### Encontra e Carrega os Processos por CNPJ ou Nome da Parte


In [22]:
# Importando as Bibliotecas
import pandas as pd
import requests 
from bs4 import BeautifulSoup
import json
import pymongo
import time
import yaml

In [36]:
# Parâmetros Iniciais TJSP Esaj
url_base = "https://esaj.tjsp.jus.br/cpopg/search.do?"

#Parâmetros de Busca 
busca_nome = "".replace(' ','+')
busca_cnpj = "33041062000109"
cod_empresa = "SULAMERICA"
dados = []

if busca_cnpj: 
    url_pesquisa = url_base + "cbPesquisa=DOCPARTE&dadosConsulta.valorConsulta=" + busca_cnpj
else: 
    url_pesquisa = url_base + "cbPesquisa=NMPARTE&dadosConsulta.valorConsulta=" + busca_nome

url_pesquisa

'https://esaj.tjsp.jus.br/cpopg/search.do?cbPesquisa=DOCPARTE&dadosConsulta.valorConsulta=33041062000109'

In [24]:
# Carrega Configurações do arquivo config
with open("../config.yaml", "r") as ymlfile:
    cfg = yaml.full_load(ymlfile)

user = cfg["mongodb"]["user"]
passwd = cfg["mongodb"]["passwd"]
host = cfg["mongodb"]["host"]
dbmongo = cfg["mongodb"]["db"]

In [25]:
# Cria Conexão com banco
url_conexao = "mongodb+srv://" + user + ":" + passwd + "@" + host + "/?retryWrites=true&w=majority"
client = pymongo.MongoClient(url_conexao)
db = client[dbmongo]


In [26]:
# Coleções Mongo
col_processos = db['processo']


In [27]:
# Lista de Foros do TJSP
lista_foros = [1, 2, 3, 4, 5, 6,7, 8, 9, 10, 11, 12,13, 14, 15, 16, 17, 18,19, 20, 21, 22, 24, 25,28, 30, 32, 35, 37, 38,40, 42, 45, 47, 48, 50,52, 53, 58, 59, 60, 62,
63, 66, 67, 68, 69, 70,71, 72, 73, 75, 76, 77,79, 80, 81, 82, 83, 84,85, 86, 87, 88, 89, 90,91, 93, 94, 95, 97, 99,100, 101, 102, 103, 104, 106,108, 111, 114, 115, 116, 118,
120, 123, 125, 126, 127, 128,129, 132, 136, 137, 140, 142,144, 145, 146, 150, 152, 153,156, 157, 159, 160, 161, 165,168, 169, 172, 176, 177, 180,185, 187, 189, 191, 196, 197,
198, 200, 201, 204, 205, 210,213, 218, 219, 220, 222, 223,224, 229, 233, 236, 238, 240,242, 244, 246, 247, 248, 252,257, 262, 263, 264, 266, 268,269, 270, 271, 272, 274, 275,
278, 279, 280, 281, 282, 283,286, 288, 291, 292, 294, 296,297, 299, 300, 301, 302, 306,309, 311, 312, 315, 318, 319,320, 322, 323, 326, 333, 334,337, 338, 341, 344, 346, 347,
348, 352, 355, 356, 357, 358,360, 361, 362, 363, 366, 368,369, 370, 372, 374, 382, 383,390, 394, 396, 397, 400, 404,405, 407, 408, 411, 412, 414,415, 416, 417, 418, 420, 424,
426, 428, 430, 431, 434, 435,438, 439, 441, 443, 444, 445,447, 449, 450, 451, 452, 453,456, 457, 458, 459, 462, 464,466, 470, 471, 472, 474, 477,480, 481, 482, 483, 484, 486,
488, 491, 493, 495, 498, 505,506, 510, 511, 512, 515, 516,523, 526, 531, 533, 534, 538,539, 541, 543, 547, 549, 553,554, 562, 563, 564, 565, 566,568, 572, 575, 576, 577, 579,
581, 582, 584, 586, 587, 588,589, 590, 595, 596, 597, 601,602, 604, 606, 607, 609, 614,615, 619, 620, 624, 625, 627,629, 634, 637, 638, 642, 646,648, 650, 651, 653, 654, 655,
659, 660, 663, 664, 665, 666,667, 668, 669, 670, 671, 672,673, 674, 675, 676, 677, 678,679, 680, 681, 682, 683, 684,685, 686, 687, 688, 689, 690,691, 692, 693, 694, 695, 696,
697, 698, 699, 700, 701, 702,703, 704, 705, 706, 707, 708,709, 710, 990]

In [28]:
def carrega_processo(linha):
    json_str = '{"processo":' + json.dumps(linha) + '}'
    col_processos.find_one_and_update({'processo.nro_processo':linha['nro_processo']},{'$set':json.loads(json_str)},upsert=True)

In [29]:
def scrapping_foro(foro):
   # Monta a URL de Busca
   url_busca = url_pesquisa + "&cdForo=" + str(foro)
   
   # Tenta fazer o request
   try:
      html = requests.get(url_busca).content
   except:
      print("Não foi possível carregar a URL")
   
   # Faz o parser do HTML
   soup = BeautifulSoup(html, 'html.parser')
   
   # Se retornou muito ou não retornou resultados, a página exibe mensagem 
   mensagem = soup.find('td',attrs={'id':'mensagemRetorno'})

   # Em alguns casos, se tive um processo só, ele retorna o processo
   processo_unico = soup.find('span',attrs={'id':'numeroProcesso'})

   # Caso retorne mais do que um processo, ele mostra a quantidades (Util para saber as páginas)
   qtde_processos = soup.find("span",attrs={'id':'contadorDeProcessos'})

   # Se não encontrou
   if mensagem: 
      mensagem = mensagem.text.strip()
      print("Scrapping do foro: " + str(foro) + " mensagem: " + mensagem)
      
   # No caso de processo Unico, faz o scrapping
   elif processo_unico:
             
      nro_processo = processo_unico.text.strip()
      print("Scrapping do foro: " + str(foro) + " processos: 1")

      classe_processo = soup.find('span',attrs={'id':'classeProcesso'}).text.strip()
      assunto_processo = soup.find('span',attrs={'id':'assuntoProcesso'})
      
      if assunto_processo: 
         assunto_processo = assunto_processo.text.strip()
      
      data_local_distribuicao = soup.find('div',attrs={'id':'dataHoraDistribuicaoProcesso'}).text.strip()

      
      linha = dict({"nro_processo":nro_processo, "link_processo":'', "nome_parte":'',"tipo_participacao":'', "classe_processo":classe_processo, "assunto_processo":assunto_processo,"data_local_distribuicao":data_local_distribuicao,"cod_empresa":cod_empresa})
      
      if linha:
         dados.append([nro_processo, '','','',classe_processo,assunto_processo,data_local_distribuicao,cod_empresa])
         carrega_processo(linha)


   # Se tem varias páginas
   elif qtde_processos:
      
      qtde = qtde_processos.text.strip()
      qtde = ''.join(c if c.isdigit() else '' for c in qtde)

      print("Scrapping do foro: " + str(foro) + " processos: " + str(qtde))
      
      paginas = round(int(qtde)/25)
      
      # Abre cada página
      for pagina in range(1,paginas+1): 
         url_pagina = url_busca + "&paginaConsulta=" + str(pagina)
         
         try:
            html_pagina = requests.get(url_pagina).content
         except:
            print("Não foi possível carregar a URL")

         soup_pagina = BeautifulSoup(html_pagina, 'html.parser')

         processo_unico_pagina = soup_pagina.find('span',attrs={'id':'numeroProcesso'})
         tabela = soup_pagina.find("div",attrs={'id':'listagemDeProcessos'})  

         # No caso de processo Unico na página, faz o scrapping
         if processo_unico_pagina:
                  
            nro_processo = processo_unico_pagina.text.strip()
            classe_processo = soup.find('span',attrs={'id':'classeProcesso'}).text.strip()
            assunto_processo = soup.find('span',attrs={'id':'assuntoProcesso'})
            
            if assunto_processo: 
               assunto_processo = assunto_processo.text.strip()
            
            data_local_distribuicao = soup.find('div',attrs={'id':'dataHoraDistribuicaoProcesso'}).text.strip()

            linha = dict({"nro_processo":nro_processo, "link_processo":'', "nome_parte":'',"tipo_participacao":'', "classe_processo":classe_processo, "assunto_processo":assunto_processo,"data_local_distribuicao":data_local_distribuicao,"cod_empresa":cod_empresa})
            
            if linha:
               dados.append([nro_processo, '','','',classe_processo,assunto_processo,data_local_distribuicao,cod_empresa])
               carrega_processo(linha)
         
         # Se não faz o scrapping a lista de processos da página
         elif tabela:

            processos_pagina = tabela.find_all('div',attrs={'class':'row unj-ai-c home__lista-de-processos'})

            for proc in processos_pagina:
               nro_processo = proc.find('a',attrs={'class':'linkProcesso'}).text.strip()
               link_processo = proc.find('a',attrs={'class':'linkProcesso'}).attrs['href'].strip()
               try: 
                  nome_parte = proc.find('div',attrs={'class':'unj-base-alt nomeParte'}).text.strip()
               except:
                  nome_parte = ''

               try: 
                  tipo_participacao = proc.find('label',attrs={'class':'unj-label tipoDeParticipacao'}).text.strip()
               except:
                  tipo_participacao = ''

               classe_processo = proc.find('div',attrs={'class':'classeProcesso'}).text.strip()
               assunto_processo = proc.find('div',attrs={'class':'assuntoPrincipalProcesso'}).text.strip()
               data_local_distribuicao = proc.find('div',attrs={'class':'dataLocalDistribuicaoProcesso'}).text.strip()

               linha = dict({"nro_processo":nro_processo, "link_processo":link_processo, "nome_parte":nome_parte, "tipo_participacao":tipo_participacao,"classe_processo":classe_processo, "assunto_processo":assunto_processo,"data_local_distribuicao":data_local_distribuicao,"cod_empresa":cod_empresa})

               if linha:
                  dados.append([nro_processo, link_processo,nome_parte,tipo_participacao,classe_processo,assunto_processo,data_local_distribuicao,cod_empresa])
                  carrega_processo(linha)


In [30]:
# Busca em Todos os Foros "-1"
#scrapping_foro(-1)

In [37]:
# Busca Foro por Foro
for foro in lista_foros: 
    scrapping_foro(foro)

Scrapping do foro: 1 processos: 44
Scrapping do foro: 2 processos: 89
Scrapping do foro: 3 processos: 59
Scrapping do foro: 4 processos: 29
Scrapping do foro: 5 processos: 22
Scrapping do foro: 6 processos: 25
Scrapping do foro: 7 processos: 23
Scrapping do foro: 8 processos: 31
Scrapping do foro: 9 processos: 36
Scrapping do foro: 10 processos: 9
Scrapping do foro: 11 processos: 78
Scrapping do foro: 12 mensagem: Não existem informações disponíveis para os parâmetros informados.
Scrapping do foro: 13 mensagem: Não existem informações disponíveis para os parâmetros informados.
Scrapping do foro: 14 processos: 8
Scrapping do foro: 15 mensagem: Não existem informações disponíveis para os parâmetros informados.
Scrapping do foro: 16 processos: 19
Scrapping do foro: 17 mensagem: Não existem informações disponíveis para os parâmetros informados.
Scrapping do foro: 18 mensagem: Não existem informações disponíveis para os parâmetros informados.
Scrapping do foro: 19 processos: 10
Scrapping do

In [38]:
from datetime import datetime
termino = datetime.now().strftime("%d/%m/%y %H:%M:%S")
print("Acabou em: " + termino)

Acabou em: 02/10/22 19:38:22


In [39]:
# Parâmetros Iniciais TJSP Esaj
url_base = "https://esaj.tjsp.jus.br/cpopg/search.do?"

#Parâmetros de Busca 
busca_nome = "".replace(' ','+')
busca_cnpj = "61573796000166"
cod_empresa = "ALLIANZ"
dados = []

if busca_cnpj: 
    url_pesquisa = url_base + "cbPesquisa=DOCPARTE&dadosConsulta.valorConsulta=" + busca_cnpj
else: 
    url_pesquisa = url_base + "cbPesquisa=NMPARTE&dadosConsulta.valorConsulta=" + busca_nome

url_pesquisa

'https://esaj.tjsp.jus.br/cpopg/search.do?cbPesquisa=DOCPARTE&dadosConsulta.valorConsulta=61573796000166'

In [40]:
# Busca Foro por Foro
for foro in lista_foros: 
    scrapping_foro(foro)

Scrapping do foro: 1 processos: 78
Scrapping do foro: 2 processos: 310
Scrapping do foro: 3 processos: 57
Scrapping do foro: 4 processos: 70
Scrapping do foro: 5 processos: 32
Scrapping do foro: 6 processos: 24
Scrapping do foro: 7 processos: 42
Scrapping do foro: 8 processos: 34
Scrapping do foro: 9 processos: 34
Scrapping do foro: 10 processos: 19
Scrapping do foro: 11 processos: 221
Scrapping do foro: 12 mensagem: Não existem informações disponíveis para os parâmetros informados.
Scrapping do foro: 13 mensagem: Não existem informações disponíveis para os parâmetros informados.
Scrapping do foro: 14 processos: 7
Scrapping do foro: 15 mensagem: Não existem informações disponíveis para os parâmetros informados.
Scrapping do foro: 16 processos: 12
Scrapping do foro: 17 mensagem: Não existem informações disponíveis para os parâmetros informados.
Scrapping do foro: 18 mensagem: Não existem informações disponíveis para os parâmetros informados.
Scrapping do foro: 19 processos: 39
Scrapping

In [35]:
from datetime import datetime
termino = datetime.now().strftime("%d/%m/%y %H:%M:%S")
print("Acabou em: " + termino)

Acabou em: 02/10/22 18:16:36


In [ ]:
df_processos = pd.DataFrame(dados,columns=["nro_processo","link_processo","nome_parte","tipo_participacao","classe_processo","assunto_processo","data_local_distribuicao","cod_empresa"])
df_processos

In [ ]:
nome_arquivo = str(time.strftime("%Y_%m_%d")) + '_tjsp_processos_' + cod_empresa.lower() + '.csv'

# Salva o arquivo 
df_processos.to_csv('./dados/' + nome_arquivo,sep=";", index=False)
